## Prepare

In [1]:
import osmnx
import geopandas as gpd
import shapely
import numpy as np
import pandas as pd

## Get sampled coordinates

In this step, I use OpenStreetMap to generate a list of coordinates from which Street View images should be requested. Taking every road segment in Old City, I sample from each segment at uniform distances, and write out the coordinates data and associated metadata for use in the next step.

In [ ]:
# Define bounding box for Old City: Front to 5th, Race to Chestnut

old_city_coords = [(-75.14822315195386, 39.95456250607422),
                   (-75.14934064485035, 39.94896593941381),
                   (-75.14238153603254, 39.948060704068624),
                   (-75.141533865292, 39.951725539856724),
                   (-75.14162161694298, 39.953938809851515)]

old_city_polygon = shapely.geometry.Polygon(old_city_coords)

In [ ]:
# Get street graph from OpenStreetMap
old_city_roads_raw = osmnx.graph.graph_from_polygon(old_city_polygon, 
                                                    network_type='drive_service')

# Add bearings for road segments
graph_with_bearing = osmnx.bearing.add_edge_bearings(old_city_roads_raw)
graph_undirected = osmnx.convert.to_undirected(graph_with_bearing)

nodes_raw, edges_raw = osmnx.graph_to_gdfs(graph_undirected)

# Filter out parking lots and duplicate segments
edges_selected = (edges_raw
                  .query('service != "driveway"')
                  .query('name.notnull()')
                  .query('osmid != [62154497]')
                  .query('osmid != [12189165]')
                  .query('osmid != [[522383581, 1360862406]]')
                  .query('osmid != [61293761]')
                  )

In [5]:
edges_selected.explore()

In [ ]:
# Sample points from segments. (0.0003 degrees ~= 33m)
points_list = [list(osmnx.utils_geo.interpolate_points(line, 0.0003)) for line in edges_selected['geometry']]

# Gather sampled points into geodataframe, and cast as geometry column
segments_gdf = edges_selected
segments_gdf['points_coords'] = points_list

points_gdf = segments_gdf.explode('points_coords')
points_gdf['geometry_points'] = [shapely.Point(point[0], point[1]) for point in points_gdf['points_coords']]

points_gdf_clean = (points_gdf
                    .drop_duplicates(subset='geometry_points', keep='first')
                    .set_geometry('geometry_points', crs = 'EPSG:4326'))

# Points look reasonable
points_gdf_clean.explore(tiles = "CartoDB positron")

For the Street View API, I will specify two bearings per point so that the vehicle is looking at the sidewalk rather than straight ahead (unless at intersections).

In [ ]:
points_bearings1 = (points_gdf_clean
                    .assign(direction = (np.round(points_gdf_clean['bearing'] + 90)) % 360)
                    [['direction', 'geometry_points']])

points_bearings2 = (points_gdf_clean
                    .assign(direction = (np.round(points_gdf_clean['bearing'] + 180)) % 360)
                    [['direction', 'geometry_points']])

points_combined = (pd.concat([points_bearings1, points_bearings2])
                   .rename_geometry('geometry'))

# Sort by coordinate values so the row indices are in a sensible order
points_output = (points_combined
                 .assign(lon = points_combined['geometry'].x)
                 .assign(lat = points_combined['geometry'].y)
                 .sort_values(by=['lat', 'lon'], ascending=[False, True])
                 .reset_index(drop=True))

points_ready = (points_output
                .assign(filename = 'file_' + points_output.index.astype(str) + '.jpg')
                [['filename', 'direction', 'lon', 'lat', 'geometry']])   

# Write out coordinates data
points_ready.to_file("data/street_view_coordinates.geojson", driver='GeoJSON')